In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import numpy as np
import time
import os
from dotenv import load_dotenv
os.environ["USE_PYGEOS"] = "0"
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
%run own_and_modified_functions.ipynb
warnings.simplefilter(action='ignore', category=FutureWarning)

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [2]:
OUTPUT_DIR = '/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data'
sdata = read_zarr( os.path.join(  OUTPUT_DIR, 'sdata.zarr'  ) )
anndata = sc.read('/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/anndataNucA11') # reset original anndata
# delete anndata
del sdata.table
# create anndata
sdata.table = anndata
sdata

SpatialData object with:
├── Images
│     ├── 'clahe': SpatialImage[cyx] (1, 12864, 10720)
│     ├── 'raw_image': SpatialImage[cyx] (1, 12864, 10720)
│     ├── 'tiling_correction': SpatialImage[cyx] (1, 12864, 10720)
│     ├── 'tophat_filtered': SpatialImage[cyx] (1, 12864, 10720)
│     └── 'transcript_density': SpatialImage[cyx] (1, 12864, 10720)
├── Labels
│     └── 'segmentation_mask': SpatialImage[yx] (12864, 10720)
├── Points
│     └── 'transcripts': DataFrame with shape: (4754932, 3) (2D points)
├── Shapes
│     ├── 'expanded_cells20': GeoDataFrame shape: (11228, 1) (2D shapes)
│     ├── 'filtered_expanded_cells20_low_counts': GeoDataFrame shape: (367, 1) (2D shapes)
│     ├── 'filtered_expanded_cells20_segmentation': GeoDataFrame shape: (50, 1) (2D shapes)
│     ├── 'filtered_expanded_cells20_size': GeoDataFrame shape: (280, 1) (2D shapes)
│     ├── 'filtered_segmentation_mask_boundaries_low_counts': GeoDataFrame shape: (367, 1) (2D shapes)
│     ├── 'filtered_segmentation_mask_

In [3]:
n_clusters = 16
leiden_mean_expression = {}
for i in range(n_clusters):
    an_cluster = sdata.table[sdata.table.obs['leiden']==str(i)]
    daf = an_cluster.to_df().mean(axis=0)
    pd.DataFrame(daf)
    leiden_mean_expression[i] = daf


In [4]:
all_mean_expression = sdata.table.to_df().mean(axis=0)

In [5]:
path_marker_genes = "/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/markerGeneListBasic.csv",
# csv to dataframe
marker_genes = pd.read_csv(path_marker_genes[0], sep=',',index_col=0)
marker_genes 

,Hepa,LSEC,Endo,Chol,Kupf,Stel,Fibr,B cell,Oth
Column1,,,,,,,,,
Acta2,0,0,0,0,0,0,1,0,1
Adamtsl2,0,0,0,0,0,1,1,0,0
Adgre1,0,0,0,0,1,0,0,0,1
Adgrg6,0,1,1,0,0,0,0,0,0
Atp6v0d2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
Vwf,0,0,1,0,0,0,0,0,0
Wnt2,0,1,1,0,0,0,0,0,0
Wnt9b,0,1,1,0,0,0,0,0,0


In [6]:
# get index marker_genes where Hepa == 1
index_hepa = marker_genes[marker_genes['Oth'] == 1].index.tolist()

In [7]:
# new dataframe
scores_leiden_celltype = pd.DataFrame()
cell_types = marker_genes.columns.tolist()
for cell_type in cell_types:
    scores_clusters = []
    for i in range(n_clusters):
        score = 0 
        for gene in marker_genes[marker_genes[cell_type] == 1].index.tolist():
            score = score + (leiden_mean_expression[i][gene] - all_mean_expression[gene])
        scores_clusters.append(score)
    scores_leiden_celltype[cell_type] = scores_clusters


In [8]:
scores_leiden_celltype 

,Hepa,LSEC,Endo,Chol,Kupf,Stel,Fibr,B cell,Oth
0,-3.371014,10.690078,1.560655,-1.691712,-1.140896,-1.698517,-2.590711,-0.356883,-2.356636
1,5.638526,-2.711275,-0.347446,2.004987,-2.094225,0.073072,-0.997631,-0.396276,-2.166421
2,-1.799640,-2.663915,-1.599521,-1.209332,-3.213873,-3.478573,-3.095252,-0.392713,-4.936969
3,5.702860,-2.544329,-0.656983,1.484638,-2.024009,-1.759808,-1.344695,-0.396470,-2.558790
4,1.880526,-2.724566,-0.593023,0.889261,-2.827720,-0.913624,-1.563831,-0.402614,-3.224753
5,3.054256,-2.626497,-1.380540,-1.269258,-3.235194,-3.658778,-3.038336,-0.392718,-4.936390
6,0.563382,-2.768325,-1.507476,-0.133823,-2.806324,-2.796269,-2.496058,-0.381625,-3.940195
7,-4.241398,0.205856,-1.790405,-2.104195,27.965250,-1.861020,-2.172533,-0.275421,10.244219
8,-3.976565,-1.810006,-0.879705,-2.131974,-0.199041,20.886359,14.207514,-0.373021,4.450577
9,3.349619,-2.758849,-1.046745,0.652369,-2.619279,-2.001096,-2.242133,-0.399653,-3.470053


In [29]:
# max column per row
sc_leiden_cellt = scores_leiden_celltype.idxmax(axis=1).to_dict()
sc_leiden_cellt = {str(k): v for k, v in sc_leiden_cellt.items()}

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [10]:
# max value per row
max_scores = scores_leiden_celltype.max(axis=1)
# second highest value per row
second_scores = scores_leiden_celltype.apply(lambda x: x.nlargest(2).values[-1], axis=1)

In [11]:
second_scores

0      1.560655
1      2.004987
2     -1.209332
3      1.484638
4      0.889261
5     -0.392718
6     -0.133823
7     10.244219
8     14.207514
9      0.652369
10    13.242090
11     6.049887
12     7.841576
13     9.222827
14    -0.915327
15    22.548402
dtype: float64

In [31]:
# cleanliness
cleanliness = (max_scores - second_scores) / ((abs(max_scores) + abs(second_scores)) / 2)
sc_leiden_cleanl = cleanliness.to_dict()
sc_leiden_cleanl = {str(k): v for k, v in sc_leiden_cleanl.items()}
sc_leiden_cleanl

{'0': 1.4904288151269707,
 '1': 0.9507511786336621,
 '2': 1.0194701248670939,
 '3': 1.1737664496068547,
 '4': 0.7157696942900167,
 '5': 2.0,
 '6': 2.0,
 '7': 0.9275727744365718,
 '8': 0.38062744748048705,
 '9': 1.347954908336663,
 '10': 0.5736342787820121,
 '11': 1.3075237452476243,
 '12': 0.6730222836863582,
 '13': 0.6940162408901234,
 '14': 2.0,
 '15': 0.21451791438660878}

In [34]:
sdata.table.obs["annotation_leiden"]=sdata.table.obs["leiden"] 
sdata.table.obs["cleanliness_leiden"]=sdata.table.obs["leiden"] 
sdata.table.obs["annotation_leiden"].replace(list(sc_leiden_cellt.keys()),list(sc_leiden_cellt.values()), inplace=True)
sdata.table.obs["cleanliness_leiden"].replace(list(sc_leiden_cleanl.keys()),list(sc_leiden_cleanl.values()), inplace=True)
sdata.table.obs

,region,instance,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_2_genes,pct_counts_in_top_5_genes,n_counts,shapeSize,leiden,annotation_leiden,cleanliness_leiden
cells,,,,,,,,,,,,,
64,1,1,12,2.564949,150,5.017280,74.666667,92.000000,150,2545.0,3,Hepa,1.173766
66,1,1,16,2.833213,38,3.663562,36.842105,68.421053,38,1257.0,11,Oth,1.307524
67,1,1,15,2.772589,50,3.931826,52.000000,76.000000,50,3057.0,10,Fibr,0.573634
68,1,1,8,2.197225,84,4.442651,80.952381,95.238095,84,2071.0,2,B cell,1.019470
69,1,1,6,1.945910,52,3.970292,67.307692,98.076923,52,1637.0,2,B cell,1.019470
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28929,1,1,8,2.197225,108,4.691348,87.037037,97.222222,108,2285.0,6,Hepa,2.000000
28944,1,1,11,2.484907,155,5.049856,71.612903,92.258065,155,1942.0,4,Hepa,0.715770
28992,1,1,10,2.397895,132,4.890349,71.969697,94.696970,132,3627.0,6,Hepa,2.000000


In [37]:
sc.tl.leiden(sdata.table, resolution=0.8, key_added='leiden_0.8')
sdata.table.obs 

,region,instance,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_2_genes,pct_counts_in_top_5_genes,n_counts,shapeSize,leiden,annotation_leiden,cleanliness_leiden,leiden_0.8
cells,,,,,,,,,,,,,,
64,1,1,12,2.564949,150,5.017280,74.666667,92.000000,150,2545.0,3,Hepa,1.173766,3
66,1,1,16,2.833213,38,3.663562,36.842105,68.421053,38,1257.0,11,Oth,1.307524,11
67,1,1,15,2.772589,50,3.931826,52.000000,76.000000,50,3057.0,10,Fibr,0.573634,10
68,1,1,8,2.197225,84,4.442651,80.952381,95.238095,84,2071.0,2,B cell,1.019470,2
69,1,1,6,1.945910,52,3.970292,67.307692,98.076923,52,1637.0,2,B cell,1.019470,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28929,1,1,8,2.197225,108,4.691348,87.037037,97.222222,108,2285.0,6,Hepa,2.000000,6
28944,1,1,11,2.484907,155,5.049856,71.612903,92.258065,155,1942.0,4,Hepa,0.715770,4
28992,1,1,10,2.397895,132,4.890349,71.969697,94.696970,132,3627.0,6,Hepa,2.000000,6
